# DGIM Implementation

The DGIM algorithm is used to count the number of "True" (1) present in a stream of 1/0 considering only the last N elements. N elements essentially constitute the sliding window. The implementation is done using a compact data structure that consumes $O(log^2(N))$ space. Explanation can be found in Chapter 4 of the book.

In the initialize function below. Intialize the following:
1. Set the error rate. Make sure that the error rate is in [0, 1]. Let c be the actual result and e be the result generated by the implmentation. Then $abs(c-e) < error\_rate * c$.

2. Now set the number of buckets (b) to be maintained in the algorithm. Set $b=1/error\_rate$. Check that the maximum number of buckets of the same size is 2.

3. The data structure to hold the buckets can be an array of queues. queue[i] will hold the timestamp of the bucket in descending order. This helps in updating the buckets and also knowing at any point of time that how many buckets of the same size there is.
        a. So, first intialize an array.
        b. Now, find out the maximum number of buckets that can be needed which is log(N) (base 2).
        c. For values computed above, initialize 1 queue each.
4. Set two variables with currentTimeStamp and oldestTimeStamp

In [1]:
##WRITE THE FUNCTION ACCORDINGLY AS INSTRUCTED
import math
from collections import deque
N=0
error_rate=0
b=0
queues=[]
currentTimeStamp=0
oldest_timestamp=0
def initialize(N, error_rate):
    if not (0 < error_rate <= 1):
            msg = ("Invalid value for error: {}. ""Rate should be between 0 and 1.".format(error_rate))
            raise ValueError(msg)
    global b
    global currentTimeStamp
    global oldest_timestamp
    b = math.ceil(1/error_rate)
    b = max(b, 2)
    global queues
    if N == 0:
        idx = -1
    else:
        idx = int(math.ceil(math.log(N)/math.log(2)))
    queues = [deque() for _ in range(idx + 1)]
    currentTimeStamp=0
    oldest_timestamp=-1

Now write the update function for adding a new element by doing the following:

1. If the new element is 1. (Do nothing for 0)
    1. Update the currentTimeStamp with a maximum value of 2*N
    2. Check if the oldest bucket needs to be removed by checking if it is too old. If it is then delete that.
3. Update the oldestTimeStamp with currentTimeStamp
4. For all the elements in the queue do the following:
    1. Add the new element
    2. pop the last two elements from the queue.
    3. Merge last two buckets if needed

In [2]:
def update(element):
    if N == 0:
        return
    global currentTimeStamp
    global oldest_timestamp
    update = (currentTimeStamp + 1) % (2 * N)
    currentTimeStamp=update
    #checking oldest bucket to be remved or not
    if (oldest_timestamp >= 0 and checkIfBucketTooOld(oldest_timestamp)):
        deleteOldestBucket()
    # if 0 is coming dont do anything
    if element is not True:
        return
    carry_over = currentTimeStamp
    #updating old tie to current time
    if oldest_timestamp == -1:
        oldest_timestamp = currentTimeStamp
    for queue in queues:
        queue.appendleft(carry_over)
        if len(queue) <= b:
            break
        x = queue.pop()#poping last element
        y = queue.pop()#poping 2nd last element
        # mergingi of two buckets starts
        carry_over = y
        if x == oldest_timestamp:
            oldest_timestamp = y

Now write the count function to count the number of elements in N and return the estimate of the number of 1s.
1. For all the queues:
    1. Find queue length
    2. For every non-null queue increment count variable with the length times the power of two (starting from 1).
2. Return half of the count computed in the step above.

In [3]:
def count():
    res = 0
    val = 0
    two_power = 1
    for queue in queues:
        queue_length = len(queue)
        if queue_length > 0:
            val = two_power
            res += queue_length * two_power
        two_power = two_power << 1
    res -= math.floor(val/2)
    return int(res)

You can write some auxilliary functions to deleteOldestBucket() and checkIfBucketTooOld()

In [4]:
def deleteOldestBucket():
    global oldest_timestamp
    for queue in reversed(queues):
        if len(queue) > 0:
            queue.pop()
            break
    oldest_timestamp = -1
    for queue in reversed(queues):
        if len(queue) > 0:
            oldest_timestamp = queue[-1]
            break

In [5]:
def checkIfBucketTooOld(bucket_timestamp):
    return (currentTimeStamp - bucket_timestamp) % (2 * N) >= N

Now make a call to initialize, make some updates and check the result that you get.

In [6]:
global N
global error_rate
N=32
error_rate=0.5
dgim = initialize(N,error_rate)
for i in range(100):
    update(True)
c = count() # This should return something like 30 when the actual count is 32.

In [7]:
print(c)

28
